In [1]:
!git clone https://github.com/phygitalism/test-tasks-3dml.git

Cloning into 'test-tasks-3dml'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 31 (delta 7), reused 22 (delta 5), pack-reused 0
Unpacking objects: 100% (31/31), done.


In [12]:
import csv
import json
import numpy as np
import torch

In [13]:
with open('test-tasks-3dml/Task1/dev.json', 'r') as f:
    data = json.load(f)

In [14]:
data[0]

{'dpdg1': 0.9571806007967657,
 'dpdg2': 0.2429918798353707,
 'id': 1,
 'x1': 0.5784676164550582,
 'x2': 0.25269502829221124}

Данную задачу можно воспринимать так: имеется граф вычислений без конечного единичного тензора, но имеются его родители и градиенты, которые приходят от этого тензора, нужно восстановить граф и посчитать производные для входа. Поскольку есть разрыв в графе, то распространение градиентов нужно запустить дважды (для двух листьев). Чтобы граф не строился заново, при первом вызове не будем его удалять, градиенты в таком случае аккумулируются - это как раз то, что нам нужно.

In [18]:
def calculate_dinput(sample):
    x1 = torch.tensor([sample['x1']], requires_grad=True)
    x2 = torch.tensor([sample['x2']], requires_grad=True)
    dpdg1 = torch.tensor([sample['dpdg1']])
    dpdg2 = torch.tensor([sample['dpdg2']])
    
    f1 = x1 + x2
    f2 = x1 * x2
    g1 = torch.tan(f1 + f2 + 100)
    g2 = f1 * f2
    
    # use one graph for two backwards 
    g1.backward(dpdg1, retain_graph=True)
    g2.backward(dpdg2)
    
    return x1.grad.item(), x2.grad.item()


Это аналитический способ вычисления производных, используем его для проверки.

In [17]:
def analytic_calc_dinput(sample):
    dpdg1 = sample['dpdg1']
    dpdg2 = sample['dpdg2']
    x1 = sample['x1']
    x2 = sample['x2']

    dg1dx1 = (x2 + 1) * (np.tan(x1 * x2 + x1 + x2 + 100)**2 + 1)
    dg2dx1 = 2 * x1 * x2 + x2**2
    dx1 = dpdg1 * dg1dx1 + dpdg2 * dg2dx1
    
    dg1dx2 = (x1 + 1) * (np.tan(x1 * x2 + x1 + x2 + 100)**2 + 1)
    dg2dx2 = 2 * x1 * x2 + x1**2
    dx2 = dpdg1 * dg1dx2 + dpdg2 * dg2dx2
    
    return dx1, dx2


Сравним результаты двух подходов:

In [36]:
eps = 1e-2
equals = []
for sample in data:
    print(f"------------------------------>{sample['id']}<------------------------------")
    auto_res = calculate_dinput(sample)
    print('Autograd result (dx1, dx2):', auto_res)
    an_res = an_calc(sample)
    print('Analytics result (dx1, dx2):', an_res, end='\n\n')
    equals.append(
        max(r1 - r2 for r1, r2 in zip(auto_res, an_res)) < eps
    )

------------------------------>1<------------------------------
Autograd result (dx1, dx2): (1.560255765914917, 2.0092973709106445)
Analytics result (dx1, dx2): (1.5602527278685892, 2.009293533654357)

------------------------------>2<------------------------------
Autograd result (dx1, dx2): (4.967437744140625, 3.0528149604797363)
Analytics result (dx1, dx2): (4.967455229688486, 3.0528259848035244)

------------------------------>3<------------------------------
Autograd result (dx1, dx2): (1397.6683349609375, 1626.5355224609375)
Analytics result (dx1, dx2): (1397.6120491079353, 1626.470009814866)

------------------------------>4<------------------------------
Autograd result (dx1, dx2): (59.70182418823242, 46.43809509277344)
Analytics result (dx1, dx2): (59.70129428536023, 46.43768117410152)

------------------------------>5<------------------------------
Autograd result (dx1, dx2): (91.28832244873047, 85.26869201660156)
Analytics result (dx1, dx2): (91.28769199037868, 85.2681000014

In [37]:
sum(equals)

9

9 из 10 случаев совпали до порядка 10^-2, в случае с неэквивалентными градиентами, значения этих градиентов довольно большие и погрешность вычислений имеет порядок 10^-1.

Ответ с помощью метода, использующего автоград.

In [38]:
answer = [['id', 'dx1', 'dx2']]
for sample in data:
    dx1, dx2 = calculate_dinput(sample)
    answer.append([sample['id'], dx1, dx2])

In [39]:
with open('answer.csv', 'w') as f:
    writer = csv.writer(f)
    for row in answer:
        writer.writerow(row)